# Assignment 1

This assignment will involve the creation of a spellchecking system and an evaluation of its performance. You may use the code snippets provided in Python for completing this or you may use the programming language or environment of your choice

Please start by downloading the corpus `holbrook.txt` from Blackboard

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    
For the purpose of this assignment you do not need to separate these words, but instead you may treat them like a single token.

*Note: you may use any functions from NLTK to complete the assignment. It should not be necessary to use other libraries and so please consult with us if your solution involves any other external library. If you use any function from NLTK in Task 6 please include a brief description of this function and how it contributes to your solution.*

## Task 1 (10 Marks)

Write a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. In the example given, there is only an error in the 10th word and so the list of indexes is [9]. It is not necessary to analyze where the error occurs inside the word.

Then split your data into a test set of 100 lines and a training set.

In [1]:
lines = open("holbrook.txt").readlines()
data = []

# my code
import nltk
import nltk.tokenize
from nltk.tokenize import RegexpTokenizer
from collections import Counter
import re

lines_clean = [l.strip() for l in lines] # get rid of /n
sentence = [nltk.word_tokenize(s) for s in lines_clean] # create a list entry for each sentence to work from

for s in sentence:
    o = []
    c = [] # create empty lists to put results into later
    i = []
    n = 0
    for w in s:
        if '|' in w:
            splt = w.split('|') # split the sentence on |
            o.append(splt[0]) # put everything on the left of | in o
            c.append(splt[1]) # put everything on the right of | in c
            i.append(n) # add the index location of | to i
        else:
            o.append(w) # if no | present then add sentence to o unchanged
            c.append(w) # if no | present then add sentence to c unchanged
        n += 1 # for every word add 1 to n until | is reached
    d = {'original': o, 'corrected': c, 'indexes': i} # put the results into the dictionary
    data.append(d) # append the dictionary to the empty data list given for us to use

assert(data[2] == {
    'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 
    'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 
    'indexes': [9]
}) # works

print(data[2])

{'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 'indexes': [9]}


The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [2]:
test = data[:100]
train = data[100:]

## **Task 2** (10 Marks): 
Calculate the frequency (number of occurrences), *ignoring case*, of all words and their unigram probability from the corrected *training* sentences.

*Hint: use `Counter` to implement this so it may be called many times*

In [3]:
# So from the question I need to be careful:
#  I only need the corrected training sentences not the whole list of training
#    Therefore need to split the training set further.
#  I also see from above its the frequency of all words, meaning I need to get rid of any special chatracters that may be counted!

In [4]:
# Background work

In [5]:
train_corrected = [i['corrected'] for i in train] # take only corrected key values
train_corrected[3] # testing line

['I',
 'saw',
 'a',
 'Royal',
 'Enfield',
 'come',
 'first',
 'it',
 'was',
 'like',
 'mine',
 '.']

In [6]:
# I found a good module in nltk for removing special characters here: https://www.nltk.org/_modules/nltk/tokenize/regexp.html and also https://towardsdatascience.com/text-processing-is-coming-c13a0e2ee15c
# So removing special characters now:
#   I can't run it directly on the list however its designed for strings
#     I can loop over the elements in the list and space them using .join()!

train_corrected = [nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(' '.join(i)) for i in train_corrected]
train_original = [i['original'] for i in train]
train_original = [nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(' '.join(i)) for i in train_original]
train_corrected[3] # checking to see if it works - yes it does as there is no full stop

['I',
 'saw',
 'a',
 'Royal',
 'Enfield',
 'come',
 'first',
 'it',
 'was',
 'like',
 'mine']

In [7]:
train_combo = train_corrected + train_original

In [8]:
# Now doing the same work on the test set for later if needed:
test_corrected = [i['corrected'] for i in test] # take only corrected key values
test_corrected[3] # testing line
test_original = [i['original'] for i in test]
test_original[3]
test_combo = test_corrected + test_original
test_combo = [nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(' '.join(i)) for i in test_combo]

In [9]:
u = [' '.join(w).lower() for w in train_corrected] # I want to change train corrected to a list of sentences so I can lower every character
u = ' '.join(u) # makes the list of sentences into one giant string
u = nltk.word_tokenize(u) # converts the giant string back into a list of single word elements
len(u)

20609

In [10]:
def unigram(word):
    # Write your code here.
    return Counter(u).get(word,0) # we only want the result of the word we input into our function not the entire dict of freq

def prob(word):
    # Write your code here.
    # I will need the sum of the words in the corpus for the specific word
    return Counter(u).get(word,0) / len(u) # return word frequency / total number of words 

# Test your code with the following
assert(unigram("me")==87) # works correctly

print(unigram('me'))
print(prob('me'))

87
0.004221456645155029


## **Task 3** (15 Marks): 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [11]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("hello", "hi"))

4


Write a function that calculates all words with *minimal* edit distance to the misspelled word. You should do this as follows

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value

*Do not implement edit distance, use the built-in NLTK function `edit_distance`*

In [12]:
def get_candidates(token):
    freq = Counter(u).most_common() # https://docs.python.org/3/library/collections.html
    unique_words = [i[0] for i in freq] # pull out words only
    dist = [edit_distance(d, token) for d in unique_words] # get the distance list of every unique word from the token
    dist_dict = dict(zip(unique_words,dist)) # dist_dict gives the edit distance for each word in the corpus
    #Now I need to order them from smallest to largest - can use sorted():
    sorted_dist_dict = {k:v for k, v in sorted(dist_dict.items(), key = lambda entry: entry[1])} # sorting by the values ascending
    smallest_dist = next(iter(sorted_dist_dict.values())) # get the smallest entry
    output_key = [x for x in sorted_dist_dict.keys() if sorted_dist_dict[x] == smallest_dist] # pull out the corresponding keys for the smallest distance for a word and put them into a list
    
    return output_key # Pulls back all words in the train_corrected list that are at smallest dist. If not at smallest dist then it just returns the word

print(get_candidates("minde"))

# Test your code as follows
assert get_candidates("minde") == ['mind', 'mine'] # works correctly

['mind', 'mine']


## Task 4 (15 Marks):

Write a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *unigram probability*. 

*Your solution to this should involve `get_candidates`*


In [13]:
def correct(sentence):
    l = []
    for word in sentence:
        # If the word is not in the frequency distribution of words in our corpus, use get_candidates to replace the wrong word with prediction
        if word not in set(u):
            l.append(get_candidates(word)[0])
        else:
            l.append(word) # else just take the word as is and assume its correct
    return l

print(correct(["this","whitr","cat"]))
assert(correct(["this","whitr","cat"]) == ['this','white','cat'])

['this', 'white', 'cat']


## **Task 5** (10 Marks): 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [14]:
def accuracy(original, prediction): 
    # I want to get the length of the set of unique words in the prediciton that are also in the actual sentence in the corpus
    # Then I want to divide that by the length of the actual set of words in the corpus for that sentence to get accuracy metric
    return len(set(correct(prediction)) & set(original)) / len(set(original))

accuracy = [accuracy(test_corrected[word], test_original[word]) for word in range(len(test_corrected))] # loop through the test dataset and create list of scores

print(f"Avg accuracy of words in each sentence in holbrook.txt: {sum(accuracy)/len(accuracy)}") # return the sum of accuracies in the list divided by the length of the list to get average accuracy
print(f"{accuracy.count(1)} sentences predicted correctly out of 100 without any error in spelling.") # count how many words were already fully correct by getting how many times a score of 1 appeared in the list of accuracies

Avg accuracy of words in each sentence in holbrook.txt: 0.6226056202855736
9 sentences predicted correctly out of 100 without any error in spelling.


Model Accuracy: **62.26%**

## **Task 6 (35 Marks):**

Consider a modification to your algorithm that would improve the accuracy of the algorithm developed in Task 3 and 4

* You may resources beyond those provided here.
* You must **not use the test data** in this task.
* Provide a short text describing what you intend to do and why. 
* Full marks for this section may be obtained without an implementation, but an implementation is preferred.
* Your implementation should not consist of more than 50 lines of code

Please note this task is marked according to: demonstration of knowledge from the lectures (10), originality and appropriateness of solution (10), completeness of description (10) and technical correctness (5)


### My Answer: Suggestions

- Going beyond unigram probability could aid in the accuracy of the model like getting bigrams probability and using them to find the likely word that should be used in the incorrect case. Could be done using interpolation (Week 4 Lectures)

    - In this case we could use a back-off method for n-gram smoothing.

- Using the `from nltk.corpus import words` module to check for correct spelling

- Using the vocab from word.net could really help the model be better: `wn.words()`

- Making sure along the way that all words are lower case when iterated over : `word.lower()`

- Making sure the word is not a digit: `word is not word.isdigit()`

- Using higher order n-gram smoothing back applying a the 'back-off' method

- Application of distributional similarity on word vectors if a similar word cannot be found in the corpus thus increasing accuracy

- Using lemmatisation and stemming (Lecture 2) 
    - Using it to identify that a word incorrectly spelled can be stemmed and returned to the correct version using the rest of the dictionaries supplied.
        - Could do a structure to say if a word in a certain criteria then stem and analyze else use lemmatisation in the cases where we need a morphological transformation to a common lemma (e.g ies,ing,ied ended words)
            - Finally from here we could even look for inflectional patterns
            
            
- Part of speech tagging of each sentence could determine which part is incorrect and return them to us for fixing with out `get_candidates()` function

- Incoroporation of even more data would be helpful like using something like Framenet, Word2Vev, GloVe, T5 or a global lexicon

- Named Entity Recognition could be used to say if the word has a tag then we know its valid and if not use `get_candidates()` (Lecture 5)

In [15]:
from nltk.tag import pos_tag

In [16]:
p = [' '.join(w).lower() for w in train_corrected]
p = ' '.join(u)
data = nltk.word_tokenize(p)

In [17]:
data = nltk.pos_tag(data)

In [18]:
data_tags = [x[1] for x in data]

In [19]:
from nltk.corpus import wordnet as wn
from nltk.corpus import words

def correct_new(sentence):
    l = []
    for word in sentence:
        # If the word (making sure its lower case) is not in the frequency dist of words in our corpus, use get_candidates to replace the wrong word with prediction
        if word.lower() not in wn.words() and word.lower() not in set(u) and word.lower() is not word.isdigit() and word.lower() not in words.words() and nltk.pos_tag(word.lower()) not in data_tags:
            l.append(get_candidates(word)[0])
        else:
            l.append(word) # else just take the word as is and assume its correct
    return l

print(correct_new(["this","whitr","cat"]))
assert(correct_new(["this","whitr","cat"]) == ['this','white','cat'])

['this', 'white', 'cat']


## **Task 7 (5 Marks):**

Repeat the evaluation (as in Task 5) of your new algorithm and show that it outperforms the algorithm from Task 3 and 4

In [20]:
def accuracy_new(original, prediction): 
    # I want to get the length of the set of unique words in the prediciton that are also in the actual sentence in the corpus training data
    # Then I want to divide that by the length of the actual set of words in the corpus for that sentence
    return len(set(correct_new(prediction)) & set(original)) / len(set(original))

accuracy_new = [accuracy_new(test_corrected[word], test_original[word]) for word in range(len(test_corrected))] # loop through the test dataset

print(f"Avg accuracy of words in each sentence: {sum(accuracy_new) / len(accuracy_new)}") 
print(f"{accuracy_new.count(1)} sentences predicted correctly out of 100 without any error in spelling.")

Avg accuracy of words in each sentence: 0.8096063291140686
17 sentences predicted correctly out of 100 without any error in spelling.


Model Accuracy: **80.96%**

In [21]:
100*((sum(accuracy_new) / len(accuracy_new)) - (sum(accuracy) / len(accuracy)))

18.700070882849495

In [22]:
100*(accuracy_new.count(1) - accuracy.count(1)) / accuracy_new.count(1)

47.05882352941177

I was able to get the model improved by **18.7%** using the proposed techniques and **47.06%** more sentences fully correctly classified. The second accuracy model takes a little longer to run due to the fact it needs to check wordnet for words.